<a href="https://colab.research.google.com/github/AyushPathak7074/ChatbotAssistant/blob/master/ChatbotAssist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#libraries needed for tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# load the intent.json files from your local device
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\r\n   "intents":[\r\n {\r\n   "tag": "greeting",\r\n   "patterns": [\r\n      "Hi",\r\n      "How are you?",\r\n      "Is anyone there?",\r\n      "Hello",\r\n      "Good day",\r\n      "What\'s up",\r\n      "how are ya",\r\n      "heyy",\r\n      "whatsup",\r\n      "??? ??? ??"\r\n   ],\r\n   "responses": [\r\n      "Hello!",\r\n      "Good to see you again!",\r\n      "Hi there, how can I help?"\r\n   ],\r\n   "context_set": ""\r\n},\r\n {\r\n   "tag": "goodbye",\r\n   "patterns": [\r\n      "cya",\r\n      "see you",\r\n      "bye bye",\r\n      "See you later",\r\n      "Goodbye",\r\n      "I am Leaving",\r\n      "Bye",\r\n      "Have a Good day",\r\n      "talk to you later",\r\n      "ttyl",\r\n      "i got to go",\r\n      "gtg"\r\n   ],\r\n   "responses": [\r\n      "Sad to see you go :(",\r\n      "Talk to you later",\r\n      "Goodbye!",\r\n      "Come back soon"\r\n   ],\r\n   "context_set": ""\r\n},\r\n {\r\n   "tag": "creator",\r\n   "patterns": [\r

In [17]:
#import our chat-bot intents file
with open('intents.json') as json_data:
  intents= json.load(json_data)

In [18]:
intents


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  